In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)


In [ ]:
df = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['Location'].value_counts()

In [ ]:
df_mel = df[df['Location'] == 'Melbourne']
df_mel.isnull().sum()

In [ ]:
df_mel.info()

In [ ]:
df_mel

In [ ]:
def impute_random_nan(df_mel, feature):
    df_mel[feature+"_random"] = df_mel[feature]
    random_sample = df_mel[feature].dropna().sample(df_mel[feature].isnull().sum(), random_state = 0) # adding random values in null data
    random_sample.index = df_mel[df_mel[feature].isnull()].index # to pick up all the indexes with null values in that particular feature
    df_mel.loc[df[feature].isnull(), feature+'_random'] = random_sample  # replace null values with random sample


In [ ]:
impute_random_nan(df_mel, 'MinTemp')


In [ ]:
impute_random_nan(df_mel, 'MaxTemp')
impute_random_nan(df_mel, 'Rainfall')
impute_random_nan(df_mel, 'Evaporation')
impute_random_nan(df_mel, 'Sunshine')
impute_random_nan(df_mel, 'Humidity9am')
impute_random_nan(df_mel, 'Humidity3pm')
impute_random_nan(df_mel, 'Pressure9am')
impute_random_nan(df_mel, 'Pressure3pm')
impute_random_nan(df_mel, 'Temp9am')
impute_random_nan(df_mel, 'Temp3pm')

In [ ]:
df_mel.isnull().sum()

In [ ]:
df_mel.drop(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm'], axis = 1, inplace = True)

In [ ]:
df_mel.isnull().sum()

In [ ]:
df_mel

In [ ]:
#Fill nulls with 0

cols = ['WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am']
for i in cols:
    df_mel[i].fillna('Missing', inplace = True)

df_mel.head()


In [ ]:
df_mel.drop(['Cloud9am', 'Cloud3pm'], axis = 1, inplace = True)
df_mel.isnull().sum()

In [ ]:
df_mel.shape

In [ ]:
df_mel = df_mel.dropna()

In [ ]:
df_mel.shape

In [ ]:
df_mel.isnull().sum()

In [ ]:
df_mel

In [ ]:
df_mel['RainToday'] = df_mel['RainToday'].map({'Yes' : 1, 'No' : 0})
df_mel['RainTomorrow'] = df_mel['RainTomorrow'].map({'Yes' : 1, 'No' : 0})

In [ ]:
df_mel

In [ ]:
# To plot the correlation chart

correlations = df_mel.corr()
f,ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlations, annot = True)


In [ ]:
#To plot pie chart

counts = df_mel['RainToday'].value_counts().sort_index()
print(counts)
# Plot a pie chart
plt.figure(figsize = (10,8))
plt.pie(df_mel['RainToday'].value_counts(), labels = df_mel['RainToday'].value_counts().index, autopct="%.1f%%")

plt.legend()
plt.show()


In [ ]:
#To plot pie chart

counts = df_mel['RainTomorrow'].value_counts().sort_index()
print(counts)
# Plot a pie chart
plt.figure(figsize = (10,8))
plt.pie(df_mel['RainTomorrow'].value_counts(), labels = df_mel['RainTomorrow'].value_counts().index, autopct="%.1f%%")

plt.legend()
plt.show()


In [ ]:
df_mel

In [ ]:
from fbprophet import Prophet

In [ ]:
#To get future prediction where Store = ref var and Sales = target var, periods = no. of periods for which prediction is desired

def pred(df_mel, periods):
    df_mel1 = df_mel[['Date', 'RainToday']].rename(columns = {'Date': 'ds', 'RainToday': 'y'}) #x axis needs to be named ds, and y axis(target var) as y for fbprophet
    df_mel1 = df_mel1.sort_values('ds')
    return df_mel1


In [ ]:
#Convert prediction list to df

lsp = pred(df_mel, 60)
dp = pd.DataFrame(lsp)
dp


# Rained or not Trend

In [ ]:
#Plotting feature for preds

plt.figure(figsize = (50,50))

def predplot(df_mel, periods):
  df_mel = df_mel[['Date', 'RainToday']].rename(columns = {'Date': 'ds', 'RainToday': 'y'}) #x axis needs to be named ds, and y axis(target var) as y for fbprophet
  df_mel = df_mel.sort_values('ds')
  model = Prophet()
  model.fit(df_mel)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure = model.plot(forecast, xlabel = 'Date', ylabel = 'RainToday')
  figure2 = model.plot_components(forecast)


predplot(df_mel, 60)


# Min Temp Trend

In [ ]:
#Plotting feature for preds

plt.figure(figsize = (50,50))

def predplot(df_mel, periods):
  df_mel = df_mel[['Date', 'MinTemp_random']].rename(columns = {'Date': 'ds', 'MinTemp_random': 'y'}) #x axis needs to be named ds, and y axis(target var) as y for fbprophet
  df_mel = df_mel.sort_values('ds')
  model = Prophet()
  model.fit(df_mel)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure = model.plot(forecast, xlabel = 'Date', ylabel = 'MinTemp_random')
  figure2 = model.plot_components(forecast)


predplot(df_mel, 60)


# Rainfall Trend

In [ ]:
#Plotting feature for preds

plt.figure(figsize = (50,50))

def predplot(df_mel, periods):
  df_mel = df_mel[['Date', 'Rainfall_random']].rename(columns = {'Date': 'ds', 'Rainfall_random': 'y'}) #x axis needs to be named ds, and y axis(target var) as y for fbprophet
  df_mel = df_mel.sort_values('ds')
  model = Prophet()
  model.fit(df_mel)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure = model.plot(forecast, xlabel = 'Date', ylabel = 'Rainfall_random')
  figure2 = model.plot_components(forecast)


predplot(df_mel, 60)


# Humidity9am Trend

In [ ]:
#Plotting feature for preds

plt.figure(figsize = (50,50))

def predplot(df_mel, periods):
  df_mel = df_mel[['Date', 'Humidity9am_random']].rename(columns = {'Date': 'ds', 'Humidity9am_random': 'y'}) #x axis needs to be named ds, and y axis(target var) as y for fbprophet
  df_mel = df_mel.sort_values('ds')
  model = Prophet()
  model.fit(df_mel)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure = model.plot(forecast, xlabel = 'Date', ylabel = 'Humidity9am_random')
  figure2 = model.plot_components(forecast)


predplot(df_mel, 60)


# Humidity3pm Trend

In [ ]:
#Plotting feature for preds

plt.figure(figsize = (50,50))

def predplot(df_mel, periods):
  df_mel = df_mel[['Date', 'Humidity3pm_random']].rename(columns = {'Date': 'ds', 'Humidity3pm_random': 'y'}) #x axis needs to be named ds, and y axis(target var) as y for fbprophet
  df_mel = df_mel.sort_values('ds')
  model = Prophet()
  model.fit(df_mel)
  future = model.make_future_dataframe(periods = periods)
  forecast = model.predict(future)
  figure = model.plot(forecast, xlabel = 'Date', ylabel = 'Humidity3pm_random')
  figure2 = model.plot_components(forecast)


predplot(df_mel, 60)
